<a href="https://colab.research.google.com/github/SaleemZYounus/Ai-Colab-Notebooks/blob/main/Play_Chopsticks_Game_with_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-generativeai

In [ ]:
import os
from google import genai

p1 = [1, 1]  # Player 1:      [right, left]
p2 = [1, 1]  # Player 2 (AI): [right, left]

def call_gemini_api(prompt):
    os.environ["Unlocker"] = "API KEY"  # PASTE API KEY HERE
    client = genai.Client(api_key=os.environ["Unlocker"])
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=prompt
    )
    output = getattr(response, "text", None) or getattr(response, "result", None) or ""
    return output.strip().lower()

def show_status():
    print("\nCurrent Game State:")
    print("  ----------------")
    print("     L     R")
    print(f"P2 [ {p2[1]} ] [ {p2[0]} ]")
    print(f"P1 [ {p1[1]} ] [ {p1[0]} ]")
    print("  ----------------\n")

def check_dead_hands():
    for i in range(2):
        if p1[i] >= 5:
            p1[i] = 0
        if p2[i] >= 5:
            p2[i] = 0

def is_game_over():
    if p1[0] == 0 and p1[1] == 0:
        print("Player 2 (AI) wins!")
        return True
    if p2[0] == 0 and p2[1] == 0:
        print("Player 1 wins!")
        return True
    return False

def can_split(player):
    total = player[0] + player[1]
    possible = []
    for i in range(total + 1):
        j = total - i
        # Exclude mirror repeats and invalid states >= 5
        if [i, j] != player and [j, i] != player and i < 5 and j < 5:
            possible.append([i, j])
    return possible

def p1_turn():
    while True:
        action = input("Your turn - do you want to 'attack' or 'split'? ").strip().lower()
        if action == 'split':
            options = can_split(p1)
            if not options:
                print("No valid split options. You must attack.")
                continue
            print("Possible splits:")
            for idx, option in enumerate(options, 1):
                print(f"{idx}: Left={option[1]}, Right={option[0]}")
            choice = input("Choose split number: ").strip()
            if choice.isdigit() and 1 <= int(choice) <= len(options):
                sel = options[int(choice) - 1]
                p1[0], p1[1] = sel[0], sel[1]
                check_dead_hands()
                print(f"You split to Left={p1[1]}, Right={p1[0]}")
                break
            else:
                print("Invalid choice.")
        elif action == 'attack':
            attacker = input("Which hand will you use? (r/l): ").strip().lower()
            attacked = input("Which hand will you attack? (r/l): ").strip().lower()
            if attacker not in 'rl' or attacked not in 'rl':
                print("Invalid input, try again.")
                continue
            att_index = 0 if attacker == 'r' else 1
            def_index = 0 if attacked == 'r' else 1
            if p1[att_index] == 0:
                print("Your attacking hand is out. Pick another.")
                continue
            if p2[def_index] == 0:
                print("Opponent hand is out. Pick another.")
                continue
            p2[def_index] += p1[att_index]
            check_dead_hands()
            break
        else:
            print("Invalid action. Choose 'attack' or 'split'.")

def parse_ai_move(move):
    move = move.replace(",", " ").lower().split()
    if not move:
        return "invalid", None, None
    if move[0] == "attack" and len(move) == 3 and move[1][0] in "rl" and move[2][0] in "rl":
        return "attack", move[1][0], move[2][0]
    if move[0] == "split" and len(move) == 3:
        try:
            val1 = int(move[1])
            val2 = int(move[2])
            return "split", val1, val2
        except: return "invalid", None, None
    return "invalid", None, None

def p2_turn():
    prompt = f"""
You are Player 2 in the Chopsticks game.
Player 1: Left={p1[1]}, Right={p1[0]}
You: Left={p2[1]}, Right={p2[0]}
Rules:
- On your turn, you can either attack one of Player 1's nonzero hands with one of your nonzero hands ('attack [your hand] [target hand]') or
- You can split your own hands between new non-redundant values which don't repeat your current or mirror state ('split [new_left] [new_right]').
- No hand may have 5 or more fingers.
Give your move exactly as: attack r l   or: split 2 2   (no other text)
Current State: Player 1: [{p1[1]},{p1[0]}], Player 2: [{p2[1]},{p2[0]}]
"""
    move = call_gemini_api(prompt)
    print(f"AI move: {move}")
    action, arg1, arg2 = parse_ai_move(move)
    if action == "attack":
        att_index = 0 if arg1 == 'r' else 1
        def_index = 0 if arg2 == 'r' else 1
        if p2[att_index] == 0 or p1[def_index] == 0:
            print("AI made an incorrect attack, skipping.")
            return
        p1[def_index] += p2[att_index]
        check_dead_hands()
        print(f"AI attacked with its {'right' if att_index==0 else 'left'} hand on your {'right' if def_index==0 else 'left'} hand.")
    elif action == "split":
        options = can_split(p2)
        if [arg2, arg1] in options:
            p2[0], p2[1] = arg2, arg1
            check_dead_hands()
            print(f"AI split to Left={p2[1]}, Right={p2[0]}")
        else:
            print("AI made an invalid split, skipping.")
    else:
        print("AI move invalid or not understood. Skipping turn.")

def main():
    show_status()
    while not is_game_over():
        p1_turn()
        show_status()
        if is_game_over():
            break
        p2_turn()
        show_status()

if __name__ == '__main__':
    main()
